In [3]:
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import cross_val_score
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import sklearn.datasets
from sklearn.model_selection import train_test_split
from xgboost import XGBRegressor
from sklearn import metrics

In [4]:
house_price_dataset=sklearn.datasets.load_boston(0)

ImportError: 
`load_boston` has been removed from scikit-learn since version 1.2.

The Boston housing prices dataset has an ethical problem: as
investigated in [1], the authors of this dataset engineered a
non-invertible variable "B" assuming that racial self-segregation had a
positive impact on house prices [2]. Furthermore the goal of the
research that led to the creation of this dataset was to study the
impact of air quality but it did not give adequate demonstration of the
validity of this assumption.

The scikit-learn maintainers therefore strongly discourage the use of
this dataset unless the purpose of the code is to study and educate
about ethical issues in data science and machine learning.

In this special case, you can fetch the dataset from the original
source::

    import pandas as pd
    import numpy as np

    data_url = "http://lib.stat.cmu.edu/datasets/boston"
    raw_df = pd.read_csv(data_url, sep="\s+", skiprows=22, header=None)
    data = np.hstack([raw_df.values[:

In [5]:
from sklearn.datasets import fetch_openml
boston=fetch_openml(name='boston',version=1,as_frame=True)
df=boston.frame
df.head()

,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT,MEDV
0,0.00632,18.0,2.31,0,0.538,6.575,65.2,4.0900,1,296.0,15.3,396.90,4.98,24.0
1,0.02731,0.0,7.07,0,0.469,6.421,78.9,4.9671,2,242.0,17.8,396.90,9.14,21.6
2,0.02729,0.0,7.07,0,0.469,7.185,61.1,4.9671,2,242.0,17.8,392.83,4.03,34.7
3,0.03237,0.0,2.18,0,0.458,6.998,45.8,6.0622,3,222.0,18.7,394.63,2.94,33.4
4,0.06905,0.0,2.18,0,0.458,7.147,54.2,6.0622,3,222.0,18.7,396.90,5.33,36.2


Features and Target

In [6]:
df['price']=boston.target
df.head()

,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT,MEDV,price
0,0.00632,18.0,2.31,0,0.538,6.575,65.2,4.0900,1,296.0,15.3,396.90,4.98,24.0,24.0
1,0.02731,0.0,7.07,0,0.469,6.421,78.9,4.9671,2,242.0,17.8,396.90,9.14,21.6,21.6
2,0.02729,0.0,7.07,0,0.469,7.185,61.1,4.9671,2,242.0,17.8,392.83,4.03,34.7,34.7
3,0.03237,0.0,2.18,0,0.458,6.998,45.8,6.0622,3,222.0,18.7,394.63,2.94,33.4,33.4
4,0.06905,0.0,2.18,0,0.458,7.147,54.2,6.0622,3,222.0,18.7,396.90,5.33,36.2,36.2


Dividing the dataset into dependent and independent features

In [7]:
X = df.iloc[:,:-1]
X = X.astype('float')
X.head()

,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT,MEDV
0,0.00632,18.0,2.31,0.0,0.538,6.575,65.2,4.0900,1.0,296.0,15.3,396.90,4.98,24.0
1,0.02731,0.0,7.07,0.0,0.469,6.421,78.9,4.9671,2.0,242.0,17.8,396.90,9.14,21.6
2,0.02729,0.0,7.07,0.0,0.469,7.185,61.1,4.9671,2.0,242.0,17.8,392.83,4.03,34.7
3,0.03237,0.0,2.18,0.0,0.458,6.998,45.8,6.0622,3.0,222.0,18.7,394.63,2.94,33.4
4,0.06905,0.0,2.18,0.0,0.458,7.147,54.2,6.0622,3.0,222.0,18.7,396.90,5.33,36.2


In [8]:
y = df['price']
y.head()
y.astype('float')

,price
0,24.0
1,21.6
2,34.7
3,33.4
4,36.2
...,...
501,22.4
502,20.6
503,23.9
504,22.0


Split the Data into Training & Test Data

In [20]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=3)
print(X.shape, X_train.shape, X_test.shape)

(506, 14) (354, 14) (152, 14)


In [10]:
print(X_train.head(),X_test.head())

         CRIM    ZN  INDUS  CHAS    NOX     RM    AGE     DIS   RAD    TAX  \
359   4.26131   0.0  18.10   0.0  0.770  6.112   81.3  2.5091  24.0  666.0   
374  18.49820   0.0  18.10   0.0  0.668  4.138  100.0  1.1370  24.0  666.0   
98    0.08187   0.0   2.89   0.0  0.445  7.820   36.9  3.4952   2.0  276.0   
273   0.22188  20.0   6.96   1.0  0.464  7.691   51.8  4.3665   3.0  223.0   
329   0.06724   0.0   3.24   0.0  0.460  6.333   17.2  5.2146   4.0  430.0   

     PTRATIO       B  LSTAT  MEDV  
359     20.2  390.74  12.67  22.6  
374     20.2  396.90  37.97  13.8  
98      18.0  393.53   3.57  43.8  
273     18.6  390.77   6.58  35.2  
329     16.9  375.21   7.34  22.6           CRIM    ZN  INDUS  CHAS    NOX     RM   AGE     DIS   RAD    TAX  \
224  0.31533   0.0   6.20   0.0  0.504  8.266  78.3  2.8944   8.0  307.0   
137  0.35233   0.0  21.89   0.0  0.624  6.454  98.4  1.8498   4.0  437.0   
453  8.24809   0.0  18.10   0.0  0.713  7.393  99.3  2.4527  24.0  666.0   
303  0.1000


Linear Regression

In [11]:
lin_reg = LinearRegression()

In [21]:
model=lin_reg.fit(X_train, y_train)

In [22]:
predicted=lin_reg.predict(X_test)

In [24]:
score1=metrics.r2_score(y_test,predicted)
print("R square:",score1)

R square: 1.0


In [25]:
lin_reg.fit(X_train,y_train)

LinearRegression()

In [26]:
y_predicted=lin_reg.predict(X_test)

In [27]:
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import cross_val_score
X=X.astype(float)
y=y.astype(float)
lin_reg=LinearRegression()
mse=cross_val_score(lin_reg,X,y,scoring='neg_mean_squared_error',cv=5)
print(mse)

[-4.13734343e-28 -7.54838350e-28 -5.92348624e-29 -7.64632722e-28
 -4.30439219e-27]


In [31]:
import pandas as pd
comparison_df=pd.DataFrame({'Actual:'})